### Dim User

In [0]:
from pyspark.sql.functions import *
import os
import sys

sys.path.append('/Workspace/Users/shwetapatel165_gmail.com#ext#@shwetapatel165gmail.onmicrosoft.com/spotify_dab')

from utils.transform import transformData


---
### AutoLoader
---

In [0]:
df_user = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format", "parquet")\
            .option("cloudFiles.schemalocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimUser/checkpoint")\
            .load("abfss://bronze@vasudev001storage.dfs.core.windows.net/DimUser")

In [0]:
df_user = df_user.withColumn("user_name", upper(col("user_name")))
df_user = df_user.withColumn("status",when(col("end_date").isNull(), "Active").otherwise("Expired"))
df_user = df_user.withColumn("start_year",year("start_date"))
df_user = df_user.withColumn("start_month",date_format(col("start_date"), "MMMM"))
df_user = df_user.drop("_rescued_data")
df_user = df_user.dropDuplicates(['user_id'])

In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimUser/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@vasudev001storage.dfs.core.windows.net/DimUser/data")\
    .toTable("spotify_cat.silver.DimUser")

In [0]:
df_track = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format", "parquet")\
            .option("cloudFiles.schemalocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimTrack/checkpoint")\
            .load("abfss://bronze@vasudev001storage.dfs.core.windows.net/DimTrack")
df_track = df_track.drop("_rescued_data")
df_track = df_track.dropDuplicates(['track_id'])
df_track = df_track.withColumn("track_classification",when(col("duration_sec") < 150, "Short")\
                    .when((col("duration_sec") >= 150) & (col("duration_sec") <= 200), "Medium")\
                    .otherwise("Long")\
                    )

df_track = df_track.withColumn("track_name",regexp_replace(col("track_name"),'-','_'))


In [0]:
df_track.writeStream.format("delta")\
    .option("checkpointLocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimTrack/checkpoint")\
    .outputMode("append")\
    .option("path","abfss://silver@vasudev001storage.dfs.core.windows.net/DimTrack/data")\
    .trigger(once=True)\
    .toTable("spotify_cat.silver.DimTrack")

In [0]:

dim_artist = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format", "parquet")\
            .option("cloudFiles.schemalocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimArtist/checkpoint")\
            .load("abfss://bronze@vasudev001storage.dfs.core.windows.net/DimArtist")
dim_artist = dim_artist.drop("_rescued_data")
dim_artist = dim_artist.dropDuplicates(['artist_id'])


In [0]:
dim_artist.writeStream.format("delta")\
    .option("checkpointLocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimArtist/checkpoint")\
    .outputMode("append")\
    .option("path","abfss://silver@vasudev001storage.dfs.core.windows.net/DimArtist/data")\
    .trigger(once=True)\
    .toTable("spotify_cat.silver.DimArtist")


In [0]:
dim_date = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format", "parquet")\
            .option("cloudFiles.schemalocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimDate/checkpoint")\
            .load("abfss://bronze@vasudev001storage.dfs.core.windows.net/DimDate")

dim_date = dim_date.drop("_rescued_data")

In [0]:
dim_date.writeStream.format("delta")\
    .option("checkpointLocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/DimDate/checkpoint")\
    .outputMode("append")\
    .option("path","abfss://silver@vasudev001storage.dfs.core.windows.net/DimDate/data")\
    .trigger(once=True)\
    .toTable("spotify_cat.silver.DimDate")

In [0]:
stream_df = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format", "parquet")\
            .option("cloudFiles.schemalocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/FactStream/checkpoint")\
            .load("abfss://bronze@vasudev001storage.dfs.core.windows.net/FactStream")

stream_df = stream_df.drop("_rescued_data")


In [0]:
stream_df.writeStream.format("delta")\
    .option("checkpointLocation", "abfss://silver@vasudev001storage.dfs.core.windows.net/FactStream/checkpoint")\
    .outputMode("append")\
    .option("path","abfss://silver@vasudev001storage.dfs.core.windows.net/FactStream/data")\
    .trigger(once=True)\
    .toTable("spotify_cat.silver.FactStream")